In [1]:
# making Data folder with os
import os
os.makedirs('Data/',exist_ok=True) 

In [2]:
# Confirm Data folder was created and files added successfully
os.listdir("Data/")

['.ipynb_checkpoints',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'title_basics.csv',
 'title_ratings_filtered.csv']

In [3]:
# import pandas and numpy library as pd
import pandas as pd 
import numpy as np

1)load the title-akas-us-only.csv file

In [4]:
# loadig akas DataFrame
akas = pd.read_csv('Data/title-akas-us-only.csv', low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


2) Load the title basics file

In [5]:
# loadig Basics DataFrame
basics = pd.read_csv('Data/title.basics.tsv.gz',sep = '\t', low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


3) Remove non-US movies from title basics

In [6]:
# Filter the basics table down to only include the US by using the filter akas dataframe
filter_us_titles = basics['tconst'].isin(akas['titleId'])
basics = basics[filter_us_titles]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"


4) Perform the remaining preprocessing steps for title basics.

In [7]:
# Convert placeholder "\N" values back to true null values to identify missing genres and runtimes.
basics = basics.replace({'\\N':np.nan})

In [8]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            0
originalTitle           0
isAdult                 0
startYear           98665
endYear           1328513
runtimeMinutes     503119
genres              28616
dtype: int64

In [9]:
# Drop rows with null values in the runtimeMinutes or genres columns ONLY
basics = basics.dropna(subset = ['runtimeMinutes','genres'])

In [10]:
# confirm rows dropping null values in the runtimeMinutes or genres columns
basics.isna().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear           8802
endYear           833124
runtimeMinutes         0
genres                 0
dtype: int64

In [11]:
# Filter to keep only full-length movies
filter_titleType = basics['titleType']== 'movie'
basics = basics[filter_titleType]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


In [12]:
# Convert startYear to a float dtype
basics['startYear'] = basics['startYear'].astype('float')

In [13]:
# Filter to keep movies with startYears that are >=2000 and <=2022
filter_startYear = (basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)
basics = basics[filter_startYear]

In [14]:
# Eliminate movies that include "Documentary" in genre
## create a filter for if the movie's value in the genres column contains the word Documentary
filter_documentaries = basics['genres'].str.contains('Documentary')
## Exclude movies in the documentary category.
basics = basics[~filter_documentaries]

5) Display a final preview of your filtered title basics and save to a csv

In [15]:
# Display basics DataFrame
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86979 entries, 34802 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86979 non-null  object 
 3   originalTitle   86979 non-null  object 
 4   isAdult         86979 non-null  object 
 5   startYear       86979 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  86979 non-null  object 
 8   genres          86979 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.6+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


The info shows that the basics DataFrame has only null values. This is logical based on the dictionnary provided (movies have only a release year). Therefore we will drop this column.

In [16]:
# Drop endYear column from basics DataFrame
basics = basics.drop('endYear', axis=1)
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86979 entries, 34802 to 10016777
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86979 non-null  object 
 3   originalTitle   86979 non-null  object 
 4   isAdult         86979 non-null  object 
 5   startYear       86979 non-null  float64
 6   runtimeMinutes  86979 non-null  object 
 7   genres          86979 non-null  object 
dtypes: float64(1), object(7)
memory usage: 6.0+ MB


In [17]:
# save basics to csv
basics.to_csv('Data/title_basics.csv', index = False)

6) Load and filter the title ratings file

In [18]:
# loadig ratings DataFrame
ratings = pd.read_csv('Data/title.ratings.tsv.gz',sep = '\t', low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1988
1,tt0000002,5.8,265
2,tt0000003,6.5,1849
3,tt0000004,5.5,178
4,tt0000005,6.2,2632


In [19]:
# Filter the ratings table to keep only movies that are included in the final title basics dataframe
filter_basics = ratings['tconst'].isin(basics['tconst'])
ratings = ratings[filter_basics]
ratings

,tconst,averageRating,numVotes
17961,tt0035423,6.4,87153
40764,tt0062336,6.4,175
46645,tt0069049,6.7,7754
63640,tt0088751,5.2,336
69953,tt0096056,5.6,846
...,...,...,...
1331411,tt9914942,6.6,178
1331437,tt9915872,6.4,9
1331450,tt9916170,7.0,7
1331451,tt9916190,3.7,243


In [20]:
# Replace "\N" with np.nan (if any)
ratings = ratings.replace({'\\N':np.nan})

7) Display a final preview of your filtered title ratings and save to a csv

In [21]:
ratings.info()
ratings

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71900 entries, 17961 to 1331462
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         71900 non-null  object 
 1   averageRating  71900 non-null  float64
 2   numVotes       71900 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


,tconst,averageRating,numVotes
17961,tt0035423,6.4,87153
40764,tt0062336,6.4,175
46645,tt0069049,6.7,7754
63640,tt0088751,5.2,336
69953,tt0096056,5.6,846
...,...,...,...
1331411,tt9914942,6.6,178
1331437,tt9915872,6.4,9
1331450,tt9916170,7.0,7
1331451,tt9916190,3.7,243


In [22]:
# save ratings to csv
ratings.to_csv('Data/title_ratings_filtered.csv', index = False)